In [1]:
import db
import networkx as nx
import pandas as pd
import psycopg2

In [2]:
from pyspark import *
from pyspark.sql import *
from graphframes import *
import findspark
import os

os.environ["JAVA_HOME"] = "/usr/local/opt/openjdk@11/"

findspark.init("./spark-3.2.0-bin-hadoop3.2")

# Start a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/15 15:40:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/15 15:40:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
tables = db.tables()
set(tables['table_name'])

{'activity_grid_activitycard',
 'activity_grid_activitypaircard',
 'activity_log_activitylog',
 'analytics_attachmenttracking',
 'analytics_event',
 'analytics_searchtracking',
 'app_config_appconfig',
 'app_config_visualtokencolor',
 'auth_group',
 'auth_group_permissions',
 'auth_permission',
 'auth_user',
 'auth_user_groups',
 'auth_user_user_permissions',
 'authtoken_token',
 'cms_slugpage',
 'data_allegation',
 'data_allegation_areas',
 'data_allegation_line_areas',
 'data_allegationcategory',
 'data_area',
 'data_attachmentfile',
 'data_attachmentfile_tags',
 'data_attachmentnarrative',
 'data_attachmentocr',
 'data_attachmentrequest',
 'data_award',
 'data_complainant',
 'data_crew',
 'data_investigator',
 'data_investigatorallegation',
 'data_involvement',
 'data_linearea',
 'data_officer',
 'data_officeralias',
 'data_officerallegation',
 'data_officerassignmentattendance',
 'data_officerbadgenumber',
 'data_officercrew',
 'data_officerhistory',
 'data_officerrank',
 'data_off

In [20]:
racepop = db.download("data_racepopulation")
racepop.shape

(517, 6)

In [22]:
salary = db.download("data_salary")
salary.shape

(209219, 14)

In [12]:
area = db.download("data_area")
area.shape

(1551, 12)

In [16]:
allegation_line_areas = db.download("data_allegation_line_areas")
allegation_line_areas.shape

(4307, 3)

In [13]:
allegation_areas = db.download("data_allegation_areas")
allegation_areas.shape

(928000, 3)

In [9]:
officerallegation = db.download("data_officerallegation")
officerallegation.shape

(249788, 15)

In [10]:
officer = db.download("data_officer")
officer.shape

(35545, 33)

In [11]:
allegation = db.download("data_allegation")
allegation.shape

(216052, 20)